## Define the dependencies

In [1]:
import json
import pandas as pd
import random
import numpy as np
import re

In [2]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

In [3]:
from configparser import ConfigParser
config = ConfigParser()
configFilePath = r'C:\configfile\keys_config.cfg'
config.read(configFilePath)

authoringKey = config.get('azure-luis', 'authoringKey')
authoringEndpoint = config.get('azure-luis', 'authoringEndpoint')
predictionKey = config.get('azure-luis', 'predictionKey')
predictionEndpoint = config.get('azure-luis', 'predictionEndpoint')

## Section 1 : import frames file

In [4]:
#convert json content to a list
fileObject = open("frames.json", "r")
jsonContent = fileObject.read()
aList = json.loads(jsonContent)
print("Total examples:", len(aList))

Total examples: 1369


### 1. Split into train and test sets

In [5]:
random.shuffle(aList)
train_list = aList[:1300]
test_list = aList[1300:]

### 2. Convert to luis app import json format

#### 2.1 get utterances

In [6]:
def convert_to_luisformat(example):

    #1. Get the text
    turn1=example['turns'][0]
    text = turn1['text']
    #2. get all arguments
    all_args = turn1['labels']['acts']

    #3. remove greetings
    names=[]
    for i in range(len(all_args)):
        name = all_args[i]['name']
        names.append(name)
        
    #combine all arguments in a list
    args=[]
    for i in range(len(all_args)):
        arg = all_args[i]['args']
        args.extend(arg)

    intent="None"
    entities=[]
    for i in range(len(args)):
        key = args[i]['key']
        #intent
        if key=='intent':
            intent=args[i]['val']
        else:
            #entities
            if 'val' in args[i]:
                val = args[i]['val']
                pos = re.search(rf'\b({val})\b', text)
                if pos !=None:
                    entity={
                          "startCharIndex": pos.start(),
                          "endCharIndex": pos.end()-1,
                          "entityName": key,
                          "children": []
                            }
                    entities.append(entity)


    #4. put all together
    return {
          "text": text,
          "intentName": intent,
          "entityLabels": entities
        }

In [7]:
train_set= [convert_to_luisformat(example) for example in train_list]

#### 2.2 get intent & entity lists

In [8]:
#1. extract unique intent values from train_set
int_list=[]
for i in range(len(train_set)):
    int_list.append(train_set[i]['intentName'])
int_val = np.unique(int_list)
print("intentNames:",int_val)
    
#3. extract unique entity values from train_set
ent_list=[]
for i in range(len(train_set)):
    ent = train_set[i]['entityLabels']
    for j in range(len(ent)):
        ent_list.append(ent[j]['entityName'])
ent_val = np.unique(ent_list)
print("entityLabels:", ent_val)

intentNames: ['None' 'book']
entityLabels: ['budget' 'category' 'count_dst_city' 'dst_city' 'end_date' 'max_duration'
 'min_duration' 'n_adults' 'n_children' 'or_city' 'seat' 'str_date']


## Section 2 : Build a LUIS application
### Authentication

In [9]:
client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

### Create a LUIS application

In [10]:
def create_app(client, appName, versionId):
    # define app basics
    appDefinition = ApplicationCreateObject (name=appName, initial_version_id=versionId, culture='en-us')

    # create app
    app_id = client.apps.add(appDefinition)

    # get app id - necessary for all other changes
    print("Created LUIS app with ID {}".format(app_id))
    
    return app_id

In [11]:
appName = "p10app" #+str(uuid.uuid4())
versionId = "0.1"

create_app(client,appName,versionId )

Created LUIS app with ID cc60ea3d-7f7c-4b89-b1da-ccc1eb3e41ce


'cc60ea3d-7f7c-4b89-b1da-ccc1eb3e41ce'

### to update after creating a new app

In [12]:
app_id= 'cc60ea3d-7f7c-4b89-b1da-ccc1eb3e41ce'

### Create intents & entities & labeled examples

In [13]:
#add intents
for intent_name in int_val:
    if intent_name != 'None':
        client.model.add_intent(app_id, versionId, intent_name)

In [14]:
# add entity to app
for entity_name in ent_val:
    client.model.add_entity(app_id, versionId, name=entity_name)

In [16]:
# Add labeled examples for the entity.
for example in train_set:
    client.examples.add(app_id, versionId, example, { "enableNestedChildren": False })

## Section 3 : Training

In [17]:
client.train.train_version(app_id, versionId)
waiting = True
while waiting:
    info = client.train.get_status(app_id, versionId)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print("The app is trained. we can now go to the LUIS portal and test it!")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
The app is trained. we can now go to the LUIS portal and test it!


## Section 4 :  Write test samples to a json file and test them in LUIS Portal

In [18]:
def convert_to_testformat(example):

    #1. Get the text
    turn1=example['turns'][0]
    text = turn1['text']
    #2. get all arguments
    all_args = turn1['labels']['acts']

    #3. remove greetings
    names=[]
    for i in range(len(all_args)):
        name = all_args[i]['name']
        names.append(name)
        
    #combine all arguments in a list
    args=[]
    for i in range(len(all_args)):
        arg = all_args[i]['args']
        args.extend(arg)

    intent="None"
    entities=[]
    for i in range(len(args)):
        key = args[i]['key']
        #intent
        if key=='intent':
            intent=args[i]['val']
        else:
            #entities
            if 'val' in args[i]:
                val = args[i]['val']
                pos = re.search(rf'\b({val})\b', text)
                if pos !=None:
                    entity={
                          "entity": key,
                          "startpos": pos.start(),
                          "endpos": pos.end()-1,
                          "children": []
                            }
                    entities.append(entity)


    #4. put all together
    return {
          "text": text,
          "intent": intent,
          "entities": entities
        }

In [19]:
test_set= [convert_to_testformat(example) for example in test_list]

In [20]:
with open('test_import.json', 'w', encoding='utf-8') as f:
    json.dump(test_set, f, ensure_ascii=False, indent=4)

## Section 5 : Publish the trained application

In [22]:
# Mark the app as public so we can query it using any prediction endpoint.
# Note: For production scenarios, you should instead assign the app to your own LUIS prediction endpoint. See:
# https://docs.microsoft.com/en-gb/azure/cognitive-services/luis/luis-how-to-azure-subscription#assign-a-resource-to-an-app
client.apps.update_settings(app_id, is_public=True)

responseEndpointInfo = client.apps.publish(app_id, versionId, is_staging=False)

In [23]:
endpoint = responseEndpointInfo.endpoint_url + \
    "?subscription-key=" + authoringKey + "&q="
print("Your app is published. You can now go to test it on\n{}".format(endpoint))

Your app is published. You can now go to test it on
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/cc60ea3d-7f7c-4b89-b1da-ccc1eb3e41ce?subscription-key=7fd25c6de2424141a15401435e068296&q=


## Section 6 : query the application for prediction

In [24]:
#query
query_text = test_set[10]['text']
print("query for prediction:", query_text)

query for prediction: Hi there! Got anything from Kabul to Tel Aviv?


### 1.  using runtime client

In [25]:
#authentication with credentials
runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)


# Production == slot name
predictionRequest = { "query" : query_text}

predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
print("Top intent: {}".format(predictionResponse.prediction.top_intent))
print("Intents: ")

for intent in predictionResponse.prediction.intents:
    print("\t{}".format (json.dumps (intent)))
print("Entities: {}".format (predictionResponse.prediction.entities))

Top intent: None
Intents: 
	"None"
Entities: {'or_city': ['Kabul'], 'dst_city': ['Tel Aviv']}


### 2. Using get request

In [26]:
import requests

luisURL="https://my-langserv.cognitiveservices.azure.com/luis/prediction/v3.0/apps/cc60ea3d-7f7c-4b89-b1da-ccc1eb3e41ce/slots/production/predict?verbose=true&show-all-intents=true&log=true&subscription-key=d5241c0c3da14d4cba68a3898dd58136&query="

r = requests.get(luisURL + query_text)
r.json()['prediction']['intents']

{'None': {'score': 0.867}, 'book': {'score': 0.7313308}}